# Creating your own GAN I

In this notebook you will learn how to create your own Generative Adversarial Network with `vegans`. This is a more advanced topic which gives you deeper insights into the design of the library. As the time of writing this notebook (2021-04-08 08:32) there are only 3 (6) GAN architectures implemented in `vegans`: `VanillaGAN`, `WasssersteinGAN`, `WassersteinGANGP` and all there conditional variants. In this notebook I will explain to you how to implement the `LSGAN` and `ConditionalLSGAN` (which will then probably be part of the library after finishing this notebook). In the next few tutorials we will create successively implement more difficult architectures (Pix2Pix, LR-GAN).

First import the usual libraries:

In [10]:
import os
import torch
import pickle

import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

os.chdir("/home/thomas/Backup/Algorithmen/GAN-pytorch")
from vegans.GAN import ConditionalWassersteinGAN, ConditionalWassersteinGANGP
from vegans.utils.utils import plot_losses, plot_images, get_input_dim

## GenerativeModel

We will first implement the unconditional variant of the `LSGAN`, investigate the base classes to be used and the move on to the conditional version.

The most important class in `vegans` is the `GenerativeModel`. It takes care of a lot of boilerplate code for logging and saving stuff, checking for the correct input and defining the correct variables. Every unconditional model (and also conditional models for that matter) should inherit from this class. It is semi-abstract in the sense that the `GenerativeModel` itself can not be used for training anything as it's missing some important functions which MUST be implemented by its children. 

These **abstract methods** are
- __init__(self, x_dim, z_dim, optim, optim_kwargs, fixed_noise_size, device, folder, ngpu):
    This takes care of the initializaton and the method 
    
    GenerativeModel.__init__(
        self, x_dim=x_dim, z_dim=z_dim, optim=optim, optim_kwargs=optim_kwargs,
        fixed_noise_size=fixed_noise_size, device=device, folder=folder, ngpu=ngpu
     )
     
     must be called at the end of the `__init__` method.
- _default_optimizer(self): returns an optimizer from torch.optim that is used if the user doesn't specify any optimizers in the `optim` keyword when constructing a class.
- _define_loss(self): Not strictly necessary but it is still kept as an abstract method so that the user has to think about what he wants to implement here. You can also implement it with a single `pass` statement. However, we will show you it's intended use here.
- calculate_losses(self, X_batch, Z_batch, who): The core function that needs to be implemented. For every batch it must populate an already existing (but empty) dictionary `self._losses`. The keys of the dictionary must include at least the keys of the used `self.neural_nets`, but can also contain other losses. We will discuss this more when we come to it.

The `GenerativeModel` will also check for the presence of one very **important** attribute:
- self.neural_nets: This is a dictionary containing all the different networks to be trained. It might look like
    {
        "Generator": generator_nn_Module,
        "Adversariat": adversariat_nn_Module,
        "Encoder": encoder_nn_Module
    }

The values of the dictionary must inherit in one way or another from `nn.Module`. The user of the implemented GAN must make sure of that by using `nn.Sequential` or building their own architectures which inherit from `nn.Module` (like shown in all previous tutorials).

The keys of the dictionary are equally as important because these will link together different parts used during training:
- self.optimizers: dict containing the same keys as `self.neural_nets`. Containing one optimizer per network.
- self.steps: dict containing the same keys as `self.neural_nets`. Containing the number of training steps per network.
- self._losses: dict containing the same keys as `self.neural_nets`. Containing the loss functions per network.

Which key will be used per training step is determined by the `who`argument of `calculate_losses`. In this example case `who` will be one of "Generator", "Adversariat" or "Encoder". All of these will be called in succession.

Now we covered most of the important things. They will be explained again at the appropriate position over the course of the next few notebooks whenever they become relevant.

## GAN1v1

We can almost start with the implementation of the LSGAN. There exists one more utility class which is not as abstract as `GenerativeModel` (it in fact inherits from `GenerativeModel`) but not yet a true `GAN` implementation. This is the `GAN1v1` which should be used whenever you want to implement a `GAN` of the structure 

self.neural_nets = {
    "Generator": generator_nn_Module,
    "Adversariat": adversariat_nn_Module
}

So one generator vs one adversariat. This includes the VanillaGAN, WassersteinGAN, WassersteinGANGP as well as the LSGAN. It already implements the `calculate_losses` abstract method (which can be overriden of course) and takes care of initialization. So implementing LSGAN becomes very easy. More advanced cases are in the next notebooks.

In [11]:
from vegans.models.unconditional.GAN1v1 import GAN1v1

## LSGAN

Now let's with the class definition and `__init__` method.

In [12]:
class LSGAN(GAN1v1):
    def __init__(
            self,
            generator,
            adversariat,
            x_dim,
            z_dim,
            optim=None,
            optim_kwargs=None,
            fixed_noise_size=32,
            device=None,
            folder="./LSGAN",
            ngpu=None):

        super().__init__(
            generator=generator, adversariat=adversariat,
            z_dim=z_dim, x_dim=x_dim, adv_type="Discriminator",
            optim=optim, optim_kwargs=optim_kwargs,
            fixed_noise_size=fixed_noise_size,
            device=device, folder=folder, ngpu=ngpu
        )

This is basically a copy of the code for the [VanillaGAN](https://github.com/tneuer/GAN-pytorch/blob/main/vegans/models/unconditional/VanillaGAN.py). We do not need to inherit from `GenerativeModel` explicitly because this is already done by `GAN1v1`.

As for all networks we expect an optim(izer) dictionary, optim_kwargs (optimizer keyword arguments), fixed_noise_size (for logging purposes), the device ("cpu" or "cuda"), folder and ngpu (number gpus). We simply pass this to the parent class [GAN1v1](https://github.com/tneuer/GAN-pytorch/blob/main/vegans/models/unconditional/GAN1v1.py) which will immediatly create the very important

`self.neural_nets = {"Generator": self.generator, "Adversariat": self.adversariat}`

You are bound by these names ("Generator", "Adversariat") if you are using `GAN1v1`. If you don't like them you need to implement a little bit more (next notebooks).

Notice that we used `adv_type="Discriminator"` which indicates that the `adversariat` must output a value between [0, 1]. This will be checked when the user passes an adversariat architecture. If you want the output to be between [-Inf, Inf] use `adv_type="Critic"`.

Because they are so simple we will implement the two missing methods for `_default_optimizer` and `_define_loss` in one go. The loss function must take two arguments:

- output from discriminator (or critic).
- real and false labels. They will be generated by the `GAN1v1` and are either arrays full of ones or zeros.

It is stated in the paper approximately as:

Discriminator: 0.5 \* ( (D(x) - b)\*\*2 (D(G(z)) - a)\*\*2 )

Generator: 0.5 \*  (D(G(z)) - c)\*\*2 

where D(x) is the discriminator output (predictions), G(z) is the generator output and a, b, c are parameters. Very often we set a=0, b=c=1. This is what we will do in our implementation.

In [15]:
class LSGAN(GAN1v1):
    def __init__(
            self,
            generator,
            adversariat,
            x_dim,
            z_dim,
            optim=None,
            optim_kwargs=None,
            fixed_noise_size=32,
            device=None,
            folder="./VanillaGAN",
            ngpu=None):

        super().__init__(
            generator=generator, adversariat=adversariat,
            z_dim=z_dim, x_dim=x_dim, adv_type="Discriminator",
            optim=optim, optim_kwargs=optim_kwargs,
            fixed_noise_size=fixed_noise_size,
            device=device, folder=folder, ngpu=ngpu
        )
        
    def _default_optimizer(self):
        return torch.optim.Adam

    def _define_loss(self):
        self.loss_functions = {"Generator": torch.nn.MSELoss(), "Adversariat": torch.nn.MSELoss()}

We chose the `torch.optim.Adam` optimizer as a default and implemented the appropriate loss. The parent classes `GAN1v1` and `GenerativeModel` will take care of all the rest. 

The network would now be ready to be used :)

But we won't stop here and go quickly over the implementation of the `ConditionalLSGAN` so we can take labels and images as conditional input.

## ConditionalLSGAN

We can basically do the same thing as before and copy from [CondtionalVanillaGAN](https://github.com/tneuer/GAN-pytorch/blob/main/vegans/models/conditional/ConditionalVanillaGAN.py). This time we will inherit from `ConditionalGAN1v1` (which inherits from `ConditionalGenerativeModel` which in turn inherits from `GenerativeModel`). Everything is a `GenerativeModel` in the end. 

The main difference is that we now must also pass the `y_dim` (Dimension of the labels).

In [16]:
from vegans.models.conditional.ConditionalGAN1v1 import ConditionalGAN1v1

class ConditionalLSGAN(ConditionalGAN1v1):
    def __init__(
            self,
            generator,
            adversariat,
            x_dim,
            z_dim,
            y_dim,
            optim=None,
            optim_kwargs=None,
            fixed_noise_size=32,
            device=None,
            folder="./ConditionalVanillaGAN",
            ngpu=None):

        super().__init__(
            generator=generator, adversariat=adversariat,
            x_dim=x_dim, z_dim=z_dim, y_dim=y_dim, adv_type="Discriminator",
            optim=optim, optim_kwargs=optim_kwargs,
            fixed_noise_size=fixed_noise_size,
            device=device, folder=folder, ngpu=ngpu
        )

    def _default_optimizer(self):
        return torch.optim.Adam

    def _define_loss(self):
        self.loss_functions = {"Generator": torch.nn.MSELoss(), "Adversariat": torch.nn.MSELoss()}

We did not need to change much at all. This algorithm should now be possible to generate specific instances of handwritten digits or even translate an image of a summer scenery into winter scenery (note that there are other special architectures for especially this last problem, like CycleGAN).